In [65]:
import pandas as pd

In [82]:
df = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50336 entries, 0 to 50335
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   도메인        50336 non-null  object
 1   카테고리       50336 non-null  object
 2   대화셋일련번호    50336 non-null  object
 3   화자         50336 non-null  object
 4   문장번호       50336 non-null  int64 
 5   고객의도       50336 non-null  object
 6   상담사의도      50336 non-null  object
 7   QA         50336 non-null  object
 8   고객질문(요청)   50336 non-null  object
 9   상담사질문(요청)  50336 non-null  object
 10  고객답변       50336 non-null  object
 11  상담사답변      50336 non-null  object
 12  개체명        50336 non-null  object
 13  용어사전       50336 non-null  object
 14  지식베이스      50336 non-null  object
dtypes: int64(1), object(14)
memory usage: 5.8+ MB


In [84]:
# 결측치 개수 확인
df.isna().sum()
# 결측치 없음

도메인          0
카테고리         0
대화셋일련번호      0
화자           0
문장번호         0
고객의도         0
상담사의도        0
QA           0
고객질문(요청)     0
상담사질문(요청)    0
고객답변         0
상담사답변        0
개체명          0
용어사전         0
지식베이스        0
dtype: int64

In [85]:
df.head()
# 결측치 있음
# -> 상세한 답변을 위해, 답변 대신 추가로 질문하는 경우가 있으므로

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"
3,다산콜센터,일반행정 문의,B2240,상담사,4,,지방세납부,Q,,어떤 은행을 이용하고 계십니까?,,,은행,은행/공공기관,"은행,공공기관"
4,다산콜센터,일반행정 문의,B2240,고객,5,지방세납부,,A,,,기업은행을 이용하고 있습니다.,,기업은행,기업은행/상호,"기업은행,상호"


---
#### 문제
1. 일반행정 데이터와 대중교통 데이터 로드
2. 두 개의 데이터프레임을 결합 (단순 행 결합)
3. 데이터 필터링 - 고객 질문에 대한 상담사 답변이 즉각적으로 오는 데이터들만 필터링
4. 중복되는 질문 데이터 제거
5. 질문들을 모아서 토큰화, 벡터화
6. 그 외의 질문 목록을 이용하여 코사인 유사도 확인 & 유사 질문과 답변 출력

In [87]:
# 2개의 데이터프레임 로드
df1 = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")
df2 = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json")

In [96]:
# 단순 행 결합 (union 결합)
# 주의) 로드 시 인덱스를 따로 지정하지 않았기 때문에, 단순 결합 시 인덱스가 중첩됨
# 따라서 Primary Key(중복 X, null X)를 유지하기 위해 ignore_index=True 필수
df_combined = pd.concat([df1, df2], axis=0, ignore_index=True)
df_combined.head()

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"
3,다산콜센터,일반행정 문의,B2240,상담사,4,,지방세납부,Q,,어떤 은행을 이용하고 계십니까?,,,은행,은행/공공기관,"은행,공공기관"
4,다산콜센터,일반행정 문의,B2240,고객,5,지방세납부,,A,,,기업은행을 이용하고 있습니다.,,기업은행,기업은행/상호,"기업은행,상호"


In [97]:
# 고객질문(요청) 컬럼의 데이터 개수 확인
df_combined['고객질문(요청)'].value_counts()
# 여러 개의 공백 데이터가 존재함을 확인함 (공백1 58725개, 공백2 78개, ...)

고객질문(요청)
                             58725
카드결제와 현금결제할 때 요금 차이가 있나요?       79
                                78
버스요금은 얼마인가요?                    55
시간은 얼마나 걸려요?                    54
                             ...  
금산 시내버스 첫차가 몇 시 인가요?             1
그럼 금산 시내버스 막차는 몇 시 인가요?          1
혹시 영동에 지하철 있나요?                  1
가평 시내버스 첫차가 몇 시 인가요?             1
그럼 가평 시내버스 막차는 몇 시 인가요?          1
Name: count, Length: 22824, dtype: int64

In [100]:
# 공백으로 이루어진 value들을 하나로 통일
# Series에서 각각의 value를 추출하여 함수에 대입 - map()

# df_combined['고객질문(요청)'].map( lambda x: x.strip() ).value_counts()
# 모든 데이터프레임의 공백값을 바꿔야 함

# df_combined.map( lambda x: x.strip() ).value_counts()
# strip()은 문자열 함수인데, 전체 데이터프레임 중엔 int 타입 데이터도 있으므로 Error

# 문자로 바꿔줌
df_combined = df_combined.map( lambda x: str(x).strip() )
df_combined.head(3)

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"


In [101]:
# 조건 설정
# 1. 현재 행의 고객질문(요청)이 ''이 아님 & 다음 행의 상담사답변이 ''이 아님
flag1 = (df_combined['고객질문(요청)'] != '') & (df_combined['상담사답변'].shift(-1) != '')

In [102]:
# 2. 현재 행의 상답사 답변이 ''이 아님 & 전 행의 고객질문(요청)이 ''이 아님
flag2 = (df_combined['상담사답변'] != '') & (df_combined['고객질문(요청)'].shift(1) != '')

In [103]:
# 각각 flag1은 고객 질문, flag2는 상담사 답변만 나옴
# flag1 조건 또는 flag2 조건 중 하나라도 True인 모든 행을 선택하고, 해당 행의 모든 컬럼 데이터를 반환
df_combined = df_combined.loc[ flag1|flag2 , ]

In [104]:
# 고객 질문 데이터 중 중복 데이터는 제거
df_combined['상담사답변'] = df_combined['상담사답변'].shift(-1)

In [105]:
# 고객질문(요청) 데이터에서 ''가 아닌 데이터만 필터
df_combined = df_combined.loc[ df_combined['고객질문(요청)'] != '' ]

In [106]:
df_combined = df_combined.drop_duplicates('고객질문(요청)').reset_index(drop=True)

In [107]:
df_combined.to_csv('민원 질의응답(즉답형데이터).csv', index=False)

In [108]:
# 토큰화, 벡터화 정의 
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [109]:
komoran = Komoran()

def tokenize(text):
    return komoran.morphs(text)

vectorizer = TfidfVectorizer(
    tokenizer=tokenize, 
    lowercase=False, 
    ngram_range= (1,1), 
    min_df = 5, 
    max_df= 0.8
)

In [110]:
# df_combined에서 고객질문(요청) 데이터를 토큰화, 백터화 작업 
X = vectorizer.fit_transform( df_combined['고객질문(요청)'].values )

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [111]:
# 질문 목록 
new_questions = [
    '여권 재발급 신청 방법을 알려줘', 
    '전입 신고가 인터넷으로 가능한가요?',
    '지방세 환급금을 어디서 신청하나요?'
]

In [ ]:
# new_questions도 토큰, 벡터화 -> fit() x
test = vectorizer.transform(new_questions)

In [113]:
# 코사인 유사도 생성 
sims = cosine_similarity(test, X)
sims

array([[0.        , 0.04846939, 0.15006351, ..., 0.        , 0.        ,
        0.        ],
       [0.02090267, 0.0209412 , 0.0619398 , ..., 0.09049193, 0.04273412,
        0.05209949],
       [0.22028995, 0.07139676, 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(3, 18952))

In [116]:
for idx, sim in enumerate(sims):
    question = new_questions[idx]
    print("유저의 질문 : ", question)
    # sim데이터에서 내림차순정렬을 한 인덱스의 목록 
    sim_idxs = sim.argsort()[::-1]
    for i in sim_idxs[:2]:
        # i : 유저의 질문에 가장 유사한 질문의 인덱스
        print(f"   유사도 : {round(sim[i], 3)}   유사 질문 : {df_combined.loc[i, '고객질문(요청)']},   답변 : {df_combined.loc[i, '상담사답변']}")

유저의 질문 :  여권 재발급 신청 방법을 알려줘
   유사도 : 0.619   유사 질문 : 신청방법을 알려주세요.,   답변 : 주민등록상 세대주와 가까운 주민센터 또는 복지로 홈페이지에서 신청가능하세요.
   유사도 : 0.568   유사 질문 : 신청방법 좀 알려주세요?,   답변 : 우선 사이트에 접속하셔서 회원가입을 해주세요. 청소년일경우 공인인증서가 없으면 본인확인절차를 거쳐 회원가입을 하고, 부모님이나 세대주분께서 가입을 하실 경우 공인인증서로 가입할 수 있습니다.
유저의 질문 :  전입 신고가 인터넷으로 가능한가요?
   유사도 : 0.738   유사 질문 : 인터넷으로도 신고가능한가요?,   답변 : 방문 접수밖에 안됩니다.
   유사도 : 0.682   유사 질문 : 인터넷으로 가능한가요?,   답변 : 인터넷으로 신청 가능합니다.
유저의 질문 :  지방세 환급금을 어디서 신청하나요?
   유사도 : 0.76   유사 질문 : 지방세 환급금 신청은 어떻게 해야하죠?,   답변 : 인터넷에서 접수를 하셔야 합니다
   유사도 : 0.566   유사 질문 : 환급금을 기부할 수도 있나요?,   답변 : 네 환급금을 사회복지공동모금회에 본인 명의로 기부가 가능합니다


---
#### 문제 2
고객 질문의 데이터를 이용하여 카테고리를 분류하는 모델을 생성
- 고객 질문 데이터들을 이용하여 토큰화, 벡터화 작업 (독립 변수)
- 타겟 데이터는 '일반행정', '대중교통' 카테고리 (종속 변수)
    - 카테고리 데이터를 LabelEncoder( ) 를 이용하여 수치화
- SVC 모델을 이용하여 벡터화된 데이터와 카테고리 데이터를 이용하여 학습
- new_questions의 카테고리를 확인

In [118]:
# 독립 변수
X = df_combined['고객질문(요청)']
# 종속 변수 
Y = df_combined['카테고리']

In [119]:
# 독립 변수를 -> 토큰화, 백터화 작업 -> SVC 모델에 학습
X_vec = vectorizer.fit_transform(X)
# 종속 변수 -> LabelEncoder()를 이용하여 문자형 데이터를 숫자형 데이터로 변환 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_le = le.fit_transform(Y)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [120]:
X_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(18952, 2111))

In [121]:
from sklearn.svm import SVC
# SVC 모델 객체를 생성
svc = SVC(
    random_state=42
)

In [123]:
svc.fit(X_vec, Y_le)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [126]:
# 질문 목록에 있는 데이터들을 토큰화, 백터화 작업을 하고 학습이 된 svc 모델을 
# 이용하여 예측 -> (0, 1) -> LabelEncoder를 이용하여 원본의 데이터로 다시 변환
new_vec = vectorizer.transform(new_questions)
pred = svc.predict(new_vec)

In [127]:
le.inverse_transform(pred)

array(['일반행정 문의', '일반행정 문의', '일반행정 문의'], dtype=object)

In [128]:
# 질문 목록 
new_questions = [
    '여권 재발급 신청 방법을 알려줘', 
    '전입 신고가 인터넷으로 가능한가요?',
    '지방세 환급금을 어디서 신청하나요?', 
    '서울역에서 영등포로 가려면 어떻게 가나요?'
]

In [129]:
# 토큰화, 백터화 
test_vec = vectorizer.transform(new_questions)
# 모델을 이용하여 예측 생성
pred2 = svc.predict(test_vec)
pred2_origin = le.inverse_transform(pred2)
pred2_origin

array(['일반행정 문의', '일반행정 문의', '일반행정 문의', '대중교통 안내'], dtype=object)

In [130]:
# test_vec -> TD-IDF 방식으로 백터화한 데이터셋 길이 (4)
# pred2_origin -> 예측 값들의 원본 데이터 길이 (4)
# 두개의 변수를 이용하여 반복문 생성 
# for idx in range(4):
#     print(test_vec[idx])
#     print(pred2_origin[idx])
#     break
for vec_data, cate in zip(test_vec, pred2_origin):
    # print(vec_data)
    # print(cate)
    # cate를 기준으로 total_df에서 카테고리 필터링 -> 백터화 작업 
    x_train = vectorizer.transform(
        df_combined.loc[ df_combined['카테고리'] == cate, '고객질문(요청)' ]
    )
    # 코사인 유사도 (질문이 하나)-> (2차원 데이터 1차원으로 변경)
    sims = cosine_similarity(vec_data, x_train).ravel()
    # 유사도를 내림차순 정렬의 형태로 인덱스의 값들을 확인 
    idxs = sims.argsort()[::-1]
    # 유사도 리스트에서 유사도가 놓은 상위 2개만 출력하여 유사 질문 답변을 출력
    for idx in idxs[:2]:
        # print(idx)
        # idx는 array의 위치 값 
        # 카테고리별로 필터링 된 데이터프레임에서 array와 같이 인덱스는 위치로 변환
        # iloc를 이용하여 유사 질문을 출력 
        print('유사도 : ', round(sims[idx], 3))
        print("유사 질문 : ", 
              df_combined.loc[df_combined['카테고리'] == cate, '고객질문(요청)'].iloc[idx]
        )
        print("답변 : ", 
              df_combined.loc[df_combined['카테고리'] == cate, '상담사답변'].iloc[idx]
        )

유사도 :  0.619
유사 질문 :  신청방법을 알려주세요.
답변 :  주민등록상 세대주와 가까운 주민센터 또는 복지로 홈페이지에서 신청가능하세요.
유사도 :  0.568
유사 질문 :  신청방법 좀 알려주세요?
답변 :  우선 사이트에 접속하셔서 회원가입을 해주세요. 청소년일경우 공인인증서가 없으면 본인확인절차를 거쳐 회원가입을 하고, 부모님이나 세대주분께서 가입을 하실 경우 공인인증서로 가입할 수 있습니다.
유사도 :  0.738
유사 질문 :  인터넷으로도 신고가능한가요?
답변 :  방문 접수밖에 안됩니다.
유사도 :  0.682
유사 질문 :  인터넷으로 가능한가요?
답변 :  인터넷으로 신청 가능합니다.
유사도 :  0.76
유사 질문 :  지방세 환급금 신청은 어떻게 해야하죠?
답변 :  인터넷에서 접수를 하셔야 합니다
유사도 :  0.566
유사 질문 :  환급금을 기부할 수도 있나요?
답변 :  네 환급금을 사회복지공동모금회에 본인 명의로 기부가 가능합니다
유사도 :  0.671
유사 질문 :  서울역에서 발산역 지하철로어떨게 가나요?
답변 :  서울역에서 공항철도지하철을 이용하셔서 김포공항에 내리시고 5호선환승하셔서 발산역에서 내리시면됩니다
유사도 :  0.571
유사 질문 :  어떻게 가나요?
답변 :  오류역에서 지하철을 탄 후 대전역 지하철에서 내리신 후 14번 버스를 타면됩니다.


In [131]:
df_combined.loc[ df_combined['카테고리'] == '일반행정 문의', '고객질문(요청)' ]

0        지방세를 내려면 어떻게 해야됩니까?
1            은행을 직접방문해도 됩니까?
2           버스로 가는 방법도 있습니까?
3             다른 납부방법도 있습니까?
4           사이트 주소가 어떻게 됩니까?
                ...         
12390        꼭 서울시에 근무해야하나요?
12391          임대료는 어떻게 되나요?
12392              보증금도 있나요?
12393             입주순위도 있나요?
12394           1순위는 누가 되나요?
Name: 고객질문(요청), Length: 12395, dtype: object